In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 4.4 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricte

In [ ]:
### IMPORTS ###
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [ ]:
DRIVER = None

In [ ]:
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

In [ ]:
initialize_driver()

Initiating driver...
Finished!


In [ ]:
DRIVER.get('https://www.vgchartz.com/analysis/platform_totals/Software/Global/')

In [ ]:
# find all class name Chart
all_item = DRIVER.find_element_by_class_name('chart')

In [ ]:
#find tbody
all_item_tbody = all_item.find_element_by_tag_name('tbody')

In [ ]:
# overview tag tbody
all_item_tbody.get_attribute('outerHTML')

'<tbody>\n            <tr>\n              <td>1</td>\n              <td><a href="http://www.vgchartz.com/gamedb/games.php?console=PS2">PlayStation 2 (PS2)</a></td>\n              <td><center>658.26</center></td>\n              <td><center>563.77</center></td>\n              <td><center>222.60</center></td>\n              <td><center>217.32</center></td>\n              <td><center>1,661.95</center></td>\n            </tr>\n            <tr>\n              <td>2</td>\n              <td><a href="http://www.vgchartz.com/gamedb/games.php?console=X360">Xbox 360 (X360)</a></td>\n              <td><center>610.56</center></td>\n              <td><center>281.93</center></td>\n              <td><center>12.90</center></td>\n              <td><center>102.64</center></td>\n              <td><center>1,008.03</center></td>\n            </tr>\n            <tr>\n              <td>3</td>\n              <td><a href="http://www.vgchartz.com/gamedb/games.php?console=PS3">PlayStation 3 (PS3)</a></td>\n       

In [ ]:
# find all tag name tr
all_item_tr = all_item_tbody.find_elements_by_tag_name('tr')
len(all_item_tr)

36

In [ ]:
# variable first element of list tr
first_all_item_tr = all_item_tr[0]

In [ ]:
# overview all item in first variable
first_all_item_tr.get_attribute('outerHTML')

'<tr>\n              <td>1</td>\n              <td><a href="http://www.vgchartz.com/gamedb/games.php?console=PS2">PlayStation 2 (PS2)</a></td>\n              <td><center>658.26</center></td>\n              <td><center>563.77</center></td>\n              <td><center>222.60</center></td>\n              <td><center>217.32</center></td>\n              <td><center>1,661.95</center></td>\n            </tr>'

In [ ]:
#get name of Platform
all_item_tr_name = first_all_item_tr.find_element_by_tag_name('a')
all_item_tr_name.text

'PlayStation 2 (PS2)'

In [ ]:
# get sale
all_item_tr_center = first_all_item_tr.find_elements_by_tag_name('center')
len(all_item_tr_center)

5

In [ ]:
# NA_sales
all_item_tr_center_NA = all_item_tr_center[0]
all_item_tr_center_NA.text

'658.26'

In [ ]:
# EU sales
all_item_tr_center_EU = all_item_tr_center[1]
all_item_tr_center_EU.text

'563.77'

In [ ]:
#Japan sales
all_item_tr_center_JP = all_item_tr_center[2]
all_item_tr_center_JP.text

'222.60'

In [ ]:
#Order_sales
all_item_tr_center_order_sale = all_item_tr_center[3]
all_item_tr_center_order_sale = all_item_tr_center_order_sale.get_attribute('innerHTML')
all_item_tr_center_order_sale

'217.32'

In [ ]:
# Global_sales
all_item_tr_center_Global = all_item_tr_center[4]
all_item_tr_center_Global = all_item_tr_center_Global.get_attribute('innerHTML')
all_item_tr_center_Global

'1,661.95'

In [ ]:
def get_product_info_single(product_item):
    d = {'Platform':'',
         'North America':'',
         'Europe':'',
         'Japan':'',
         'Order_Sales':'',
         'Global':''
         }

    # name get Platform through find_element_by_class_name
    try:
        all_item_tr_name = product_item.find_element_by_tag_name('a')
        d['Platform'] = all_item_tr_name.text
    except NoSuchElementException:
        pass
    try:
      all_item_tr_center = product_item.find_elements_by_tag_name('center')
    except NoSuchElementException:
        pass
    # get NA from .text
    try:
        all_item_tr_center_NA = all_item_tr_center[0]
        d['North America'] = all_item_tr_center_NA.text
    except NoSuchElementException:
      pass
    try:
        all_item_tr_center_EU = all_item_tr_center[1]
        d['Europe'] = all_item_tr_center_EU.text
    except NoSuchElementException:
        pass
    
    # get japan sales from .text
    try:
        all_item_tr_center_JP = all_item_tr_center[2]
        d['Japan'] = all_item_tr_center_JP.text
    except NoSuchElementException:
        pass
        
    
    # get order sale from text
    try:
        all_item_tr_center_order_sale = all_item_tr_center[3]
        all_item_tr_center_order_sale = all_item_tr_center_order_sale.get_attribute('innerHTML')
        d['Order_Sales'] = all_item_tr_center_order_sale
    except NoSuchElementException:
        pass
    try:
        all_item_tr_center_Global = all_item_tr_center[4]
        all_item_tr_center_Global = all_item_tr_center_Global.get_attribute('innerHTML')
        d['Global'] = all_item_tr_center_Global
    except NoSuchElementException:
        pass
    
    return d

In [ ]:
# use for loop get all element of url
data_2 =  []
for i in range(len(all_item_tr)):
  all_item_tr_platform = all_item_tr[i]
  temp = get_product_info_single(all_item_tr_platform)
  data_2.append(temp)

In [ ]:
# Export to csv
df = pd.DataFrame(data = data_2, columns=data_2[0].keys())
df.to_csv('VGchart_console_software_sales.csv', encoding='utf-8-sig')